<a href="https://colab.research.google.com/github/AngelSalmones/escritura_IA_System/blob/develop/Limpio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se comienza cargando la GPU para nuetro proyecto.

In [0]:
# Check if we have GPU available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Instalamos la database emnist en el caso que que no se tenga.

In [0]:
# Installing emnist dataset in case we do not have it
!pip install -q emnist

Importamos las librerias que necesita nuestro proyecto.

In [0]:
# Imports
from emnist import extract_training_samples, extract_test_samples
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import keras
import random

Se declaran variables.

In [0]:
# Variables
batch_size = 128
num_classes = 62
epochs = 5

Declaramos las dimensiones de nuestras imagenes.

In [0]:
# Input image dimensions
img_rows, img_cols = 28, 28

Separamos nuestro dataset entre valores de entrenamiento y test.

In [0]:
# The data, split between train and test sets
images_digits_training, labels_digits_training = extract_training_samples('digits')
images_digits_test, labels_digits_test = extract_test_samples('digits')

images_letters_training, labels_letters_training = extract_training_samples('letters')
images_letters_test, labels_letters_test = extract_test_samples('letters')

imagenes_prueba = np.copy(images_digits_training)
labels_prueba = np.copy(labels_digits_training)

ind = list(range(240000))

random.shuffle(ind)

imagenes_copia = images_digits_training[ind]

labels_copia = labels_digits_training[ind]

imagenes_copia = imagenes_copia[:2400]
labels_copia = labels_copia[:2400]

labels_letters_training = labels_letters_training + 9
labels_letters_test = labels_letters_test + 9

images_training = np.concatenate((imagenes_copia, images_letters_training))
images_test = np.concatenate((images_digits_test, images_letters_test))
labels_training = np.concatenate((labels_copia, labels_letters_training))
labels_test = np.concatenate((labels_digits_test, labels_letters_test))

Pregunta la forma de las imagrnes y en el caso necesario las transforma.


In [0]:
# Reshaping images acording to the format in keras
if K.image_data_format() == 'channels_first':
    images_training = images_training.reshape(images_training.shape[0], 1, img_rows, img_cols)
    images_test = images_test.reshape(images_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    images_training = images_training.reshape(images_training.shape[0], img_rows, img_cols, 1)
    images_test = images_test.reshape(images_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Normaliza a las imagenes a numeros flotantes de [0,1].

In [0]:
images_training = images_training.astype('float32')
images_test = images_test.astype('float32')
images_training /= 255
images_test /= 255
print('images_training shape:', images_training.shape)
print(images_training.shape[0], 'train samples')
print(images_test.shape[0], 'test samples')

images_training shape: (127200, 28, 28, 1)
127200 train samples
60800 test samples


Convierte los vectores a matrices.

In [0]:
# Convert class vectors to binary class matrices
labels_training = keras.utils.to_categorical(labels_training, num_classes)
labels_test = keras.utils.to_categorical(labels_test, num_classes)

Se crea el modelo.

In [0]:
# Creating the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
#Cosas que yo modifique 
model.add(Conv2D(256, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
######
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Entrenamos el modelo.

In [0]:
#Training the model
model.fit(images_training, labels_training,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(images_test, labels_test))

Train on 127200 samples, validate on 60800 samples
Epoch 1/5
127200/127200 [==============================] - 102s 802us/step - loss: 0.8356 - acc: 0.7670 - val_loss: 1.8730 - val_acc: 0.5225
Epoch 2/5
127200/127200 [==============================] - 101s 793us/step - loss: 0.3726 - acc: 0.8884 - val_loss: 1.5495 - val_acc: 0.5777
Epoch 3/5
127200/127200 [==============================] - 101s 792us/step - loss: 0.3119 - acc: 0.9061 - val_loss: 1.2012 - val_acc: 0.6147
Epoch 4/5
127200/127200 [==============================] - 100s 790us/step - loss: 0.2793 - acc: 0.9138 - val_loss: 1.3703 - val_acc: 0.6119
Epoch 5/5
127200/127200 [==============================] - 100s 790us/step - loss: 0.2607 - acc: 0.9187 - val_loss: 1.3775 - val_acc: 0.6248


Evaluamos el modelo.

In [0]:
# Model Score
val_loss, val_acc = model.evaluate(images_test, labels_test)
predictions = model.predict([images_test])
print(np.argmax(predictions[1]))
print(images_test[1].shape)
plt.imshow(images_test[1][:,:,0], cmap='binary')
plt.show()

print('Test loss:', val_loss)
print('Test accuracy:', val_acc)

Guardamos la molecula.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'RedNeuronal/Models/'
model.save(base_dir+"siquiero.model")
#model.save("siquiero.model")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
